# Build an Agent

https://python.langchain.com/docs/tutorials/agents/

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [2]:
from langchain_community.llms.mlx_pipeline import MLXPipeline

llm = MLXPipeline.from_model_id(
    "mlx-community/Qwen2.5-32B-Instruct-4bit",
    pipeline_kwargs={"max_tokens": 4000, "temp": 0.1},
)

from langchain_community.chat_models.mlx import ChatMLX
model = ChatMLX(llm=llm)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [3]:
os.environ["TAVILY_API_KEY"] = "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"

In [15]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
search_results

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1735087316, 'localtime': '2024-12-24 16:41'}, 'current': {'last_updated_epoch': 1735086600, 'last_updated': '2024-12-24 16:30', 'temp_c': 12.9, 'temp_f': 55.3, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 12.1, 'wind_kph': 19.4, 'wind_degree': 288, 'wind_dir': 'WNW', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 79, 'cloud': 0, 'feelslike_c': 11.2, 'feelslike_f': 52.2, 'windchill_c': 11.2, 'windchill_f': 52.2, 'heatindex_c': 12.9, 'heatindex_f': 55.3, 'dewpoint_c': 9.3, 'dewpoint_f': 48.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.2, 'gust_mph': 16.7, 'gust_kph': 26.9}}"},
 {'url': 'https://www.peoplesweather.com/weather/

In [13]:
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

In [5]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.


'Hello! How can I assist you today?'

In [6]:
model_with_tools = model.bind_tools(tools)

In [7]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
ContentString: Hello! How can I assist you today?
ToolCalls: []


In [8]:
response = model_with_tools.invoke([HumanMessage(content="Use the search tool to search the weather in SF today.")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

[Warning] Specifying sampling arguments to ``generate_step`` is deprecated. Pass in a ``sampler`` instead.
ContentString: I'm currently unable to perform real-time web searches or access live data. However, you can easily check the weather in San Francisco today by using a weather website or app, such as the Weather Channel, AccuWeather, or a service like Google Weather. Simply enter "San Francisco weather today" into the search bar of your choice to get the most up-to-date information.
ToolCalls: []
